In [38]:
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from futils import LIGS
from rdkit.Geometry import Point3D
mols = list(Chem.SDMolSupplier(LIGS, sanitize = False))
mols.pop(1187)
mols.pop(1051)
with open("site_centers.txt") as file:
    centers = eval(file.read())
pos = lambda mol: mol.GetConformer().GetPositions()

[10:08:26] Warning: molecule is tagged as 3D, but all Z coords are zero


In [ ]:
sites = [1, 2, 3, 5, 7, 8]
for site in sites:
    transforms = [centers[3] - np.average(mol.GetConformer().GetPositions(), axis = 0,
    # weights = [atom.GetMass() for atom in mol.GetAtoms()]
    ) for mol in mols]
    for mol, trans in zip(mols, transforms):
        conf = mol.GetConformer()
        positions = conf.GetPositions()
        for i, (x,y,z) in enumerate(positions + trans):
            conf.SetAtomPosition(i, Point3D(x,y,z))
    with Chem.SDWriter(f"to_{site}.sdf") as writer:
        [writer.write(mol) for mol in mols]

In [20]:
import pandas as pd
import numpy as np
# import sdf_to_parquet
# sdf_to_parquet.extract_single_file("site_3_full_gnina.sdf", "site_3_full_gnina.parquet")

In [55]:
df = pd.read_parquet("site_3_full_gnina.parquet")
df[["CNN_VS", "CNNscore", "CNNaffinity"]] = df[["CNN_VS", "CNNscore", "CNNaffinity"]].astype(float)

In [31]:
idk = []
for name, group in df.groupby("name"):

    idk.append((np.diff(group[["CNN_VS", "CNNscore", "CNNaffinity"]], axis = 0) < 0).all(axis = 0))

In [36]:
df[~df["name"].duplicated()]

,NumLipinskiHBA,Converged,Energy,NumLipinskiHBD,minimizedAffinity,CNNscore,CNN_VS,CNNaffinity_variance,CNNaffinity,name,GENERIC_NAME,SlogP,DATABASE_ID,ExactMW
0,7,true,27.92390752372882,3,-5.64720,0.725090,4.228119,5.2659673691,5.831162,Pyridoxal Phosphate,Pyridoxal Phosphate,0.51742,DB00114,247.02457367000002
9,3,true,18.677580752525788,3,-5.53561,0.588226,2.931149,4.0929093361,4.983034,L-Phenylalanine,L-Phenylalanine,0.6409999999999996,DB00120,165.078978592
18,5,true,35.8735544015155,3,-5.58757,0.475177,2.948742,3.6445305347,6.205571,Biotin,Biotin,0.7967999999999997,DB00121,244.088163372
27,3,true,8.94301825774931,3,-4.81232,0.587424,3.263804,4.9708147049,5.556133,Methionine,Methionine,0.15139999999999998,DB00134,149.051049592
36,4,true,17.017301229313578,4,-6.10490,0.627994,3.293745,2.3102805614,5.244864,L-Tyrosine,L-Tyrosine,0.3465999999999996,DB00135,181.073893212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,8,true,38.12098305801024,4,-6.80635,0.662087,4.175922,8.5092658997,6.307207,Menadiol diphosphate,Menadiol diphosphate,2.09122,DB14650,334.000740604
10890,5,true,21.04554419862575,3,-5.21621,0.583499,3.019501,7.7163314819,5.174815,Chlorphenesin carbamate,Chlorphenesin carbamate,1.58907,DB14656,245.045485544
10899,6,true,87.71466781982704,2,-5.39019,0.436724,2.643380,1.1225302219,6.052741,Paramethasone acetate,Paramethasone acetate,2.3224,DB14657,434.2104669319999
10908,8,true,95.4360052542094,4,-4.87349,0.619674,3.723505,0.7716251612,6.008813,Betamethasone phosphate,Betamethasone phosphate,2.012700000000001,DB14669,472.16623276999985


In [39]:
supp = Chem.SDMolSupplier("site_3_full_gnina.sdf")
mols = [supp[i*9] for i in range(len(supp)//9)]
